In [1]:
import pandas as pd
import os

In [2]:
current_path = os.getcwd()
print(current_path)

c:\Users\sepujas\Dev\mat\notebooks


In [3]:
base_path = os.path.abspath(os.path.join(current_path, os.pardir))
print(base_path)

c:\Users\sepujas\Dev\mat


In [4]:
vl06o_file_path = os.path.join(base_path, 'data', 'raw', 'tbl_vl06o.txt')
print(vl06o_file_path)
vl06o_preprocessed_path = os.path.join(base_path, 'data', 'preprocessed', 'tbl_vl06o.csv')
print(vl06o_preprocessed_path)

c:\Users\sepujas\Dev\mat\data\raw\tbl_vl06o.txt
c:\Users\sepujas\Dev\mat\data\preprocessed\tbl_vl06o.csv


In [5]:
df_vl06o = pd.read_csv(vl06o_file_path, sep='\t', skiprows=3, encoding='latin1')

In [6]:
unnamed_columns = [col for col in df_vl06o.columns if 'Unnamed:' in col]
df_vl06o.drop(columns=unnamed_columns, inplace=True)
column_titles = df_vl06o.columns.tolist()
new_column_titles = {col: col.strip().replace(' ', '_').replace('-','_').replace('.','') for col in column_titles}
df_vl06o.rename(columns=new_column_titles, inplace=True)

In [7]:
column_counts = df_vl06o.columns.value_counts()
duplicate_columns = column_counts[column_counts > 1].index

In [8]:
for col in duplicate_columns:
    col_indices = [i for i, x in enumerate(df_vl06o.columns) if x == col]
    for j, index in enumerate(col_indices):
        df_vl06o.columns.values[index] = f"{col}_{j+1}"

In [9]:
df_vl06o.columns.tolist()

['SOrg',
 'Delivery',
 'Item',
 'Material',
 'Item_Description',
 'DlvQty',
 'SU',
 'Batch',
 'OPS',
 'PickingSts',
 'WM',
 'GM',
 'Route',
 'SpStck_No',
 'Sold_to',
 'Name_of_Sold_to_Party',
 'Pur_Doc',
 'TrpPlanDt']

In [10]:
df_vl06o.head()

,SOrg,Delivery,Item,Material,Item_Description,DlvQty,SU,Batch,OPS,PickingSts,WM,GM,Route,SpStck_No,Sold_to,Name_of_Sold_to_Party,Pur_Doc,TrpPlanDt
0,3000,326105590,10,412689,DX cartucho 6.8/11 M10 STD pardo,100.0,PC,NaN,NaN,NaN,NaN,A,ZMX998,NaN,18469073.0,COPIISA OFFSHORE,574106548,12.06.2024
1,3000,326105591,10,412689,DX cartucho 6.8/11 M10 STD pardo,100.0,PC,NaN,NaN,NaN,NaN,A,ZMX998,NaN,18469073.0,COPIISA OFFSHORE,574184151,12.06.2024
2,3000,326108615,10,2210796,DX cartucho 6.8/11 M10 rojo,6.0,PC,NaN,NaN,NaN,NaN,A,ZMX998,NaN,16623943.0,ADOLFO GERARDO AYALA ROSAS,574202314,13.06.2024
3,8300,326493667,10,2267010,Lâmina serra de sabre SP 20 1014 (5) uni,5.0,PC,NaN,NaN,NaN,NaN,A,ZBR888,NaN,18722793.0,ANGLOGOLD ASHANTI - LAMEGO,574223455,15.07.2024
4,8300,326516675,10,2267010,Lâmina serra de sabre SP 20 1014 (5) uni,5.0,PC,NaN,NaN,NaN,NaN,A,ZBR888,NaN,18722793.0,ANGLOGOLD ASHANTI - LAMEGO,574223455,15.07.2024


In [11]:
df_vl06o.dtypes

SOrg                       int64
Delivery                   int64
Item                       int64
Material                   int64
Item_Description          object
DlvQty                   float64
SU                        object
Batch                    float64
OPS                      float64
PickingSts               float64
WM                       float64
GM                        object
Route                     object
SpStck_No                float64
Sold_to                  float64
Name_of_Sold_to_Party     object
Pur_Doc                    int64
TrpPlanDt                 object
dtype: object

In [12]:
def transform_columns(df):
    str_columns = [
        'Item_Description', 'SU', 'GM' ,'Route', 'Name_of_Sold_to_Party', 'Pur_Doc']
    for col in str_columns:
        df[col] = df[col].astype(str).str.strip()

    numeric_columns = ['SOrg', 'Delivery', 'Item', 'Material', 'Batch', 'OPS', 'PickingSts', 'WM' , 'SpStck_No' ,'Sold_to']
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int).astype(str)

    df['DlvQty'] = df['DlvQty'].astype(str).str.replace('.', '').str.replace(',', '.').astype(float)

    date_columns = ['TrpPlanDt']
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors="coerce", format='%d.%m.%Y')

transform_columns(df_vl06o)

In [13]:
df_vl06o.head()

,SOrg,Delivery,Item,Material,Item_Description,DlvQty,SU,Batch,OPS,PickingSts,WM,GM,Route,SpStck_No,Sold_to,Name_of_Sold_to_Party,Pur_Doc,TrpPlanDt
0,3000,326105590,10,412689,DX cartucho 6.8/11 M10 STD pardo,1000.0,PC,0,0,0,0,A,ZMX998,0,18469073,COPIISA OFFSHORE,574106548,2024-06-12
1,3000,326105591,10,412689,DX cartucho 6.8/11 M10 STD pardo,1000.0,PC,0,0,0,0,A,ZMX998,0,18469073,COPIISA OFFSHORE,574184151,2024-06-12
2,3000,326108615,10,2210796,DX cartucho 6.8/11 M10 rojo,60.0,PC,0,0,0,0,A,ZMX998,0,16623943,ADOLFO GERARDO AYALA ROSAS,574202314,2024-06-13
3,8300,326493667,10,2267010,Lâmina serra de sabre SP 20 1014 (5) uni,50.0,PC,0,0,0,0,A,ZBR888,0,18722793,ANGLOGOLD ASHANTI - LAMEGO,574223455,2024-07-15
4,8300,326516675,10,2267010,Lâmina serra de sabre SP 20 1014 (5) uni,50.0,PC,0,0,0,0,A,ZBR888,0,18722793,ANGLOGOLD ASHANTI - LAMEGO,574223455,2024-07-15
